In [20]:
import pandas as pd
import plotly.express as px
import requests

# Fetch data function remains the same
def fetch_openalex_data(concept_id):
    url = "https://api.openalex.org/works"
    params = {
        'filter': f'concepts.id:{concept_id}',
        'per-page': 200
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

# Updated process_data function
def process_data(data):
    works = data.get('results', [])
    processed = []
    for work in works:
        title = work.get('title', 'Unknown Title')
        publication_year = work.get('publication_year', 'Unknown Year')
        citation_count = work.get('cited_by_count', 0)
        processed.append({
            'Title': title,
            'Publication Year': publication_year,
            'Citations': citation_count
        })
    df = pd.DataFrame(processed)
    # Ensure no missing or duplicate entries
    df = df.dropna(subset=['Publication Year', 'Title'])
    df = df.groupby(['Publication Year', 'Title'], as_index=False).sum()
    return df

# Updated create_treemap function
def create_treemap(df, file_path=""):
    df['Publication Year'] = df['Publication Year'].astype(str)  # Ensure it's a string
    fig = px.treemap(
        df,
        path=['Publication Year', 'Title'],  # Hierarchical path
        values='Citations',  # Metric to size the boxes
        title='Tree Map of OpenAlex Data (Filtered by Technology)',
        color='Citations',
        color_continuous_scale='Viridis'
    )

    fig.update_layout(
    font=dict(
        family='Helvetica', 
        size=14,         # Font size
        color='black'     # Font color
    ),
    title_x=0.5  # Center the title
)
    fig.show()

        # Save the figure as an HTML file
    if file_path:
        fig.write_html(file_path)


# Main execution
if __name__ == "__main__":
    concept_id = "C154945302"  # Replace with your concept ID
    data = fetch_openalex_data(concept_id)
    df = process_data(data) 
    create_treemap(df, file_path = 'opeanlex_citations_treemap.html') 